Load pkl file in this new worksheet

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
root = '/content/drive/MyDrive/Andreas personal project'

In [ ]:
import joblib
loaded_model = joblib.load('{}/football-match-prediction-model.pkl'.format(root))

In [ ]:
def result(row):
    if row['GF'] == row['GA']:
        res = 'Draw'
    elif row['GF'] > row['GA'] and row['Team Venue'] == 'Home' or row['GF'] < row['GA'] and row['Team Venue'] == 'Away':
        res = 'Home Win'
    else:
        res = 'Away Win'
    return res

We need the features for the latest batch of predictions. First load the historical data we have


In [ ]:
import pandas as pd
import numpy as np

def myfunction(team):
  df_team = pd.read_csv(team)
  df_team = df_team.drop(columns=['Time', 'Day', 'Gls', 'SoT','SoT%','G/Sh','G/SoT','Dist','FK','PK','PKatt','npxG','npxG/Sh','G-xG','np:G-xG','Match Report'])
  df_team.dropna(inplace = True)
  df_team.drop(index=df_team[df_team['Sh'] == 'Sh'].index, inplace=True)
  df_team['Filename'] = os.path.basename(team)
  df_team[['Team','Extra']] = df_team['Filename'].str.split('_', expand=True)
  df_team = df_team.drop(columns=['Filename', 'Extra'])
  df_team.rename(columns={'Venue':'Team Venue', 'Result': 'Result(For Team)'},inplace=True)
  df_team['Match Result'] = df_team.apply(result, axis=1)
  df_team.insert(4, 'Match Result', df_team.pop('Match Result'))
  df_team.insert(2, 'Team', df_team.pop('Team'))
  df_team['cumulative_avg_xG'] = df_team['xG'].expanding().mean()
  df_team['cumulative_avg_GF'] = df_team['GF'].expanding().mean()
  df_team['cumulative_avg_GA'] = df_team['GA'].expanding().mean()
  df_team['cumulative_avg_shots'] = df_team['Sh'].expanding().mean()
  df_team['avg_xG_last_3'] = df_team['xG'].rolling(3).mean()
  df_team['avg_GF_last_3']= df_team['GF'].rolling(3).mean()
  df_team['avg_GA_last_3']= df_team['GA'].rolling(3).mean()
  df_team['avg_Sh_last_3']= df_team['Sh'].rolling(3).mean()
  df_team.rename(columns={'xG':'match_xG','GF':'match_GF','GA':'match_GA','Sh':'match_Sh'}, inplace=True)
  df_team_last=df_team.tail(1)
  return df_team_last
  #return df_team

In [ ]:
import glob
import os

path = '{}/Team data'.format(root)

all_files = glob.glob(os.path.join(path , "*2025-26.csv"))

team_temp = []
print('Teams loaded:')
for filename in all_files:
    df = myfunction(filename)
    team_temp.append(df)
    print(filename)
df_allteam = pd.concat(team_temp, axis=0, ignore_index=True)
df_allteam

Teams loaded:
/content/drive/MyDrive/Andreas personal project/Team data/Arsenal_2025-26.csv
/content/drive/MyDrive/Andreas personal project/Team data/Aston Villa_2025-26.csv
/content/drive/MyDrive/Andreas personal project/Team data/Bournemouth_2025-26.csv
/content/drive/MyDrive/Andreas personal project/Team data/Brentford_2025-26.csv
/content/drive/MyDrive/Andreas personal project/Team data/Brighton_2025-26.csv
/content/drive/MyDrive/Andreas personal project/Team data/Burnley_2025-26.csv
/content/drive/MyDrive/Andreas personal project/Team data/Chelsea_2025-26.csv
/content/drive/MyDrive/Andreas personal project/Team data/Crystal Palace_2025-26.csv
/content/drive/MyDrive/Andreas personal project/Team data/Everton_2025-26.csv
/content/drive/MyDrive/Andreas personal project/Team data/Fulham_2025-26.csv
/content/drive/MyDrive/Andreas personal project/Team data/Leeds United_2025-26.csv
/content/drive/MyDrive/Andreas personal project/Team data/Liverpool_2025-26.csv
/content/drive/MyDrive/And

,Date,Round,Team,Team Venue,Result(For Team),Match Result,match_GF,match_GA,Opponent,match_Sh,match_xG,cumulative_avg_xG,cumulative_avg_GF,cumulative_avg_GA,cumulative_avg_shots,avg_xG_last_3,avg_GF_last_3,avg_GA_last_3,avg_Sh_last_3
0,31/08/2025,Matchweek 3,Arsenal,Away,L,Home Win,0,1,Liverpool,11,0.5,1.500000,2.000000,0.333333,12.333333,1.500000,2.000000,0.333333,12.333333
1,31/08/2025,Matchweek 3,Aston Villa,Home,L,Away Win,0,3,Crystal Palace,13,1.1,0.833333,0.000000,1.333333,11.000000,0.833333,0.000000,1.333333,11.000000
2,30/08/2025,Matchweek 3,Bournemouth,Away,W,Away Win,1,0,Tottenham,20,1.5,1.500000,1.333333,1.333333,14.666667,1.500000,1.333333,1.333333,14.666667
3,30/08/2025,Matchweek 3,Brentford,Away,L,Home Win,1,2,Sunderland,6,1.2,1.266667,1.000000,1.666667,8.000000,1.266667,1.000000,1.666667,8.000000
4,31/08/2025,Matchweek 3,Brighton,Home,W,Home Win,2,1,Manchester City,11,2.2,2.033333,1.000000,1.333333,10.666667,2.033333,1.000000,1.333333,10.666667
5,30/08/2025,Matchweek 3,Burnley,Away,L,Home Win,2,3,Manchester Utd,6,1.2,1.033333,1.333333,2.000000,9.000000,1.033333,1.333333,2.000000,9.000000
6,30/08/2025,Matchweek 3,Chelsea,Home,W,Home Win,2,0,Fulham,12,2.1,2.133333,2.333333,0.333333,14.333333,2.133333,2.333333,0.333333,14.333333
7,31/08/2025,Matchweek 3,Crystal Palace,Away,W,Away Win,3,0,Aston Villa,5,2.6,1.433333,1.333333,0.333333,8.000000,1.433333,1.333333,0.333333,8.000000
8,30/08/2025,Matchweek 3,Everton,Away,W,Away Win,3,2,Wolves,10,1.9,1.433333,1.666667,1.000000,9.333333,1.433333,1.666667,1.000000,9.333333
9,30/08/2025,Matchweek 3,Fulham,Away,L,Home Win,0,2,Chelsea,11,0.9,1.100000,0.666667,1.333333,10.333333,1.100000,0.666667,1.333333,10.333333


Function to return conditional values

In [ ]:
def result2(row):
    if row['homeGF'] == row['awayGF']:
        res = 'Draw'
    elif row['homeGF'] > row['awayGF']:
        res = 'Home Win'
    else:
        res = 'Away Win'
    return res

Drop columns that we do not need for fixtures

In [ ]:
df_allteam = df_allteam.drop(columns=['Date','Round','Team Venue', 'Result(For Team)','Match Result','Opponent'])
df_allteam

,Team,match_GF,match_GA,match_Sh,match_xG,cumulative_avg_xG,cumulative_avg_GF,cumulative_avg_GA,cumulative_avg_shots,avg_xG_last_3,avg_GF_last_3,avg_GA_last_3,avg_Sh_last_3
0,Arsenal,0,1,11,0.5,1.500000,2.000000,0.333333,12.333333,1.500000,2.000000,0.333333,12.333333
1,Aston Villa,0,3,13,1.1,0.833333,0.000000,1.333333,11.000000,0.833333,0.000000,1.333333,11.000000
2,Bournemouth,1,0,20,1.5,1.500000,1.333333,1.333333,14.666667,1.500000,1.333333,1.333333,14.666667
3,Brentford,1,2,6,1.2,1.266667,1.000000,1.666667,8.000000,1.266667,1.000000,1.666667,8.000000
4,Brighton,2,1,11,2.2,2.033333,1.000000,1.333333,10.666667,2.033333,1.000000,1.333333,10.666667
5,Burnley,2,3,6,1.2,1.033333,1.333333,2.000000,9.000000,1.033333,1.333333,2.000000,9.000000
6,Chelsea,2,0,12,2.1,2.133333,2.333333,0.333333,14.333333,2.133333,2.333333,0.333333,14.333333
7,Crystal Palace,3,0,5,2.6,1.433333,1.333333,0.333333,8.000000,1.433333,1.333333,0.333333,8.000000
8,Everton,3,2,10,1.9,1.433333,1.666667,1.000000,9.333333,1.433333,1.666667,1.000000,9.333333
9,Fulham,0,2,11,0.9,1.100000,0.666667,1.333333,10.333333,1.100000,0.666667,1.333333,10.333333


Next, we want to load the next gameweek's matches (Date, Home Team, Away Team)

In [ ]:
df_gwfix = pd.read_csv('{}/2025 26 Fixtures/Gameweek 4 fixtures.csv'.format(root))
df_gwfix = df_gwfix.drop(columns=['Wk', 'Day','Time', 'Attendance','Venue','Referee','Match Report','Notes'])
df_gwfix.rename(columns={'Home' : 'Home Team', 'Away' : 'Away Team'}, inplace=True)
df_gwfix

,Date,Home Team,Away Team
0,13/09/2025,Arsenal,Nott'ham Forest
1,13/09/2025,Crystal Palace,Sunderland
2,13/09/2025,Everton,Aston Villa
3,13/09/2025,Fulham,Leeds United
4,13/09/2025,Newcastle Utd,Wolves
5,13/09/2025,Bournemouth,Brighton
6,13/09/2025,West Ham,Tottenham
7,13/09/2025,Brentford,Chelsea
8,14/09/2025,Burnley,Liverpool
9,14/09/2025,Manchester City,Manchester Utd


Split fixture df into home and away (To make the merging easier)

In [ ]:
df_gwfixhome = df_gwfix.drop(columns=['Away Team'])

We want to replace the team names with their encoded value in order to have the data in a form where we can make a prediction

In [ ]:
df_teamsenc = pd.read_csv('{}/Encode tables/teamencoding.csv'.format(root))
df_teamsenc

,Team,Team encoded
0,Arsenal,0
1,Aston Villa,1
2,Bournemouth,2
3,Brentford,3
4,Brighton,4
5,Burnley,5
6,Chelsea,6
7,Crystal Palace,7
8,Everton,8
9,Fulham,9


In [ ]:
df_teamsenc.loc['26'] = ['Sunderland', 26]
df_teamsenc


,Team,Team encoded
0,Arsenal,0
1,Aston Villa,1
2,Bournemouth,2
3,Brentford,3
4,Brighton,4
5,Burnley,5
6,Chelsea,6
7,Crystal Palace,7
8,Everton,8
9,Fulham,9


In [ ]:
df_gwfixhomemerged = pd.merge(df_gwfixhome, df_allteam, left_on=  ['Home Team'],
                   right_on= ['Team'], how='left')
df_gwfixhomemerged = df_gwfixhomemerged.drop(columns='Team')
df_gwfixhomemerged = pd.merge(df_teamsenc, df_gwfixhomemerged, left_on=['Team'],right_on=['Home Team'], how='right')
df_gwfixhomemerged = df_gwfixhomemerged.drop(columns='Team')
df_gwfixhomemerged.rename(columns={'Team encoded':'Home Team encoded', 'match_GF':'home_team_match_GF', 'match_GA':'home_team_match_GA', 'match_Sh':'home_team_match_Sh', 'match_xG':'home_team_match_xG','cumulative_avg_xG': 'home_team_cumulative_avg_xG', 'cumulative_avg_GF' : 'home_team_cumulative_avg_GF', 'cumulative_avg_GA':'home_team_cumulative_avg_GA', 'cumulative_avg_shots': 'home_team_cumulative_avg_shots','avg_xG_last_3':'home_team_avg_xG_last_3', 'avg_GF_last_3':'home_team_avg_GF_last_3','avg_GA_last_3':'home_team_avg_GA_last_3', 'avg_Sh_last_3':'home_team_avg_Sh_last_3'}, inplace=True)
df_gwfixhomemerged

,Home Team encoded,Date,Home Team,home_team_match_GF,home_team_match_GA,home_team_match_Sh,home_team_match_xG,home_team_cumulative_avg_xG,home_team_cumulative_avg_GF,home_team_cumulative_avg_GA,home_team_cumulative_avg_shots,home_team_avg_xG_last_3,home_team_avg_GF_last_3,home_team_avg_GA_last_3,home_team_avg_Sh_last_3
0,0,13/09/2025,Arsenal,0,1,11,0.5,1.500000,2.000000,0.333333,12.333333,1.500000,2.000000,0.333333,12.333333
1,7,13/09/2025,Crystal Palace,3,0,5,2.6,1.433333,1.333333,0.333333,8.000000,1.433333,1.333333,0.333333,8.000000
2,8,13/09/2025,Everton,3,2,10,1.9,1.433333,1.666667,1.000000,9.333333,1.433333,1.666667,1.000000,9.333333
3,9,13/09/2025,Fulham,0,2,11,0.9,1.100000,0.666667,1.333333,10.333333,1.100000,0.666667,1.333333,10.333333
4,17,13/09/2025,Newcastle Utd,0,0,8,0.5,0.966667,0.666667,1.000000,11.333333,0.966667,0.666667,1.000000,11.333333
5,2,13/09/2025,Bournemouth,1,0,20,1.5,1.500000,1.333333,1.333333,14.666667,1.500000,1.333333,1.333333,14.666667
6,24,13/09/2025,West Ham,3,0,11,2.4,1.233333,1.333333,2.666667,11.666667,1.233333,1.333333,2.666667,11.666667
7,3,13/09/2025,Brentford,1,2,6,1.2,1.266667,1.000000,1.666667,8.000000,1.266667,1.000000,1.666667,8.000000
8,5,14/09/2025,Burnley,2,3,6,1.2,1.033333,1.333333,2.000000,9.000000,1.033333,1.333333,2.000000,9.000000
9,15,14/09/2025,Manchester City,1,2,12,1.8,1.933333,1.666667,1.333333,12.333333,1.933333,1.666667,1.333333,12.333333


In [ ]:
df_gwfixawaymerged = pd.merge(df_gwfix, df_allteam, left_on=  ['Away Team'],
                   right_on= ['Team'], how='left')
df_gwfixawaymerged = df_gwfixawaymerged.drop(columns=['Team'])
df_gwfixawaymerged = pd.merge(df_teamsenc, df_gwfixawaymerged, left_on=['Team'],right_on=['Away Team'], how='right')
df_gwfixawaymerged = df_gwfixawaymerged.drop(columns='Team')
df_gwfixawaymerged.rename(columns={'Team encoded':'Away Team encoded','match_GF':'away_team_match_GF', 'match_GA':'away_team_match_GA', 'match_Sh':'away_team_match_Sh', 'match_xG':'away_team_match_xG','cumulative_avg_xG': 'away_team_cumulative_avg_xG', 'cumulative_avg_GF' : 'away_team_cumulative_avg_GF', 'cumulative_avg_GA':'away_team_cumulative_avg_GA', 'cumulative_avg_shots': 'away_team_cumulative_avg_shots','avg_xG_last_3':'away_team_avg_xG_last_3', 'avg_GF_last_3':'away_team_avg_GF_last_3','avg_GA_last_3':'away_team_avg_GA_last_3', 'avg_Sh_last_3':'away_team_avg_Sh_last_3'}, inplace=True)
df_gwfixawaymerged

,Away Team encoded,Date,Home Team,Away Team,away_team_match_GF,away_team_match_GA,away_team_match_Sh,away_team_match_xG,away_team_cumulative_avg_xG,away_team_cumulative_avg_GF,away_team_cumulative_avg_GA,away_team_cumulative_avg_shots,away_team_avg_xG_last_3,away_team_avg_GF_last_3,away_team_avg_GA_last_3,away_team_avg_Sh_last_3
0,19,13/09/2025,Arsenal,Nott'ham Forest,0,3,11,0.7,1.133333,1.333333,1.666667,10.333333,1.133333,1.333333,1.666667,10.333333
1,26,13/09/2025,Crystal Palace,Sunderland,2,1,11,1.5,1.000000,1.666667,1.000000,10.000000,1.000000,1.666667,1.000000,10.000000
2,1,13/09/2025,Everton,Aston Villa,0,3,13,1.1,0.833333,0.000000,1.333333,11.000000,0.833333,0.000000,1.333333,11.000000
3,11,13/09/2025,Fulham,Leeds United,0,0,10,0.7,0.966667,0.333333,1.666667,11.000000,0.966667,0.333333,1.666667,11.000000
4,25,13/09/2025,Newcastle Utd,Wolves,2,3,12,1.1,0.733333,0.666667,2.666667,9.000000,0.733333,0.666667,2.666667,9.000000
5,4,13/09/2025,Bournemouth,Brighton,2,1,11,2.2,2.033333,1.000000,1.333333,10.666667,2.033333,1.000000,1.333333,10.666667
6,22,13/09/2025,West Ham,Tottenham,0,1,5,0.2,1.200000,1.666667,0.333333,11.000000,1.200000,1.666667,0.333333,11.000000
7,6,13/09/2025,Brentford,Chelsea,2,0,12,2.1,2.133333,2.333333,0.333333,14.333333,2.133333,2.333333,0.333333,14.333333
8,13,14/09/2025,Burnley,Liverpool,1,0,9,0.5,1.133333,2.666667,1.333333,11.000000,1.133333,2.666667,1.333333,11.000000
9,16,14/09/2025,Manchester City,Manchester Utd,3,2,25,3.6,2.233333,1.333333,1.333333,18.666667,2.233333,1.333333,1.333333,18.666667


In [ ]:
df_gwfixallmerged = pd.merge(df_gwfixhomemerged,df_gwfixawaymerged, on=['Date','Home Team'])
df_gwfixallmerged

,Home Team encoded,Date,Home Team,home_team_match_GF,home_team_match_GA,home_team_match_Sh,home_team_match_xG,home_team_cumulative_avg_xG,home_team_cumulative_avg_GF,home_team_cumulative_avg_GA,...,away_team_match_Sh,away_team_match_xG,away_team_cumulative_avg_xG,away_team_cumulative_avg_GF,away_team_cumulative_avg_GA,away_team_cumulative_avg_shots,away_team_avg_xG_last_3,away_team_avg_GF_last_3,away_team_avg_GA_last_3,away_team_avg_Sh_last_3
0,0,13/09/2025,Arsenal,0,1,11,0.5,1.500000,2.000000,0.333333,...,11,0.7,1.133333,1.333333,1.666667,10.333333,1.133333,1.333333,1.666667,10.333333
1,7,13/09/2025,Crystal Palace,3,0,5,2.6,1.433333,1.333333,0.333333,...,11,1.5,1.000000,1.666667,1.000000,10.000000,1.000000,1.666667,1.000000,10.000000
2,8,13/09/2025,Everton,3,2,10,1.9,1.433333,1.666667,1.000000,...,13,1.1,0.833333,0.000000,1.333333,11.000000,0.833333,0.000000,1.333333,11.000000
3,9,13/09/2025,Fulham,0,2,11,0.9,1.100000,0.666667,1.333333,...,10,0.7,0.966667,0.333333,1.666667,11.000000,0.966667,0.333333,1.666667,11.000000
4,17,13/09/2025,Newcastle Utd,0,0,8,0.5,0.966667,0.666667,1.000000,...,12,1.1,0.733333,0.666667,2.666667,9.000000,0.733333,0.666667,2.666667,9.000000
5,2,13/09/2025,Bournemouth,1,0,20,1.5,1.500000,1.333333,1.333333,...,11,2.2,2.033333,1.000000,1.333333,10.666667,2.033333,1.000000,1.333333,10.666667
6,24,13/09/2025,West Ham,3,0,11,2.4,1.233333,1.333333,2.666667,...,5,0.2,1.200000,1.666667,0.333333,11.000000,1.200000,1.666667,0.333333,11.000000
7,3,13/09/2025,Brentford,1,2,6,1.2,1.266667,1.000000,1.666667,...,12,2.1,2.133333,2.333333,0.333333,14.333333,2.133333,2.333333,0.333333,14.333333
8,5,14/09/2025,Burnley,2,3,6,1.2,1.033333,1.333333,2.000000,...,9,0.5,1.133333,2.666667,1.333333,11.000000,1.133333,2.666667,1.333333,11.000000
9,15,14/09/2025,Manchester City,1,2,12,1.8,1.933333,1.666667,1.333333,...,25,3.6,2.233333,1.333333,1.333333,18.666667,2.233333,1.333333,1.333333,18.666667


In [ ]:
df_gwfixallmerged['home_team_avg_xG_last_3'] = df_gwfixallmerged['home_team_avg_xG_last_3'].fillna(df_gwfixallmerged['home_team_cumulative_avg_xG'])
df_gwfixallmerged['home_team_avg_GF_last_3'] = df_gwfixallmerged['home_team_avg_GF_last_3'].fillna(df_gwfixallmerged['home_team_cumulative_avg_GF'])
df_gwfixallmerged['home_team_avg_GA_last_3'] = df_gwfixallmerged['home_team_avg_GA_last_3'].fillna(df_gwfixallmerged['home_team_cumulative_avg_GA'])
df_gwfixallmerged['home_team_avg_Sh_last_3'] = df_gwfixallmerged['home_team_avg_Sh_last_3'].fillna(df_gwfixallmerged['home_team_cumulative_avg_shots'])
df_gwfixallmerged['away_team_avg_xG_last_3'] = df_gwfixallmerged['away_team_avg_xG_last_3'].fillna(df_gwfixallmerged['away_team_cumulative_avg_xG'])
df_gwfixallmerged['away_team_avg_GF_last_3'] = df_gwfixallmerged['away_team_avg_GF_last_3'].fillna(df_gwfixallmerged['away_team_cumulative_avg_GF'])
df_gwfixallmerged['away_team_avg_GA_last_3'] = df_gwfixallmerged['away_team_avg_GA_last_3'].fillna(df_gwfixallmerged['away_team_cumulative_avg_GA'])
df_gwfixallmerged['away_team_avg_Sh_last_3'] = df_gwfixallmerged['away_team_avg_Sh_last_3'].fillna(df_gwfixallmerged['away_team_cumulative_avg_shots'])
df_gwfixallmerged

,Home Team encoded,Date,Home Team,home_team_match_GF,home_team_match_GA,home_team_match_Sh,home_team_match_xG,home_team_cumulative_avg_xG,home_team_cumulative_avg_GF,home_team_cumulative_avg_GA,...,away_team_match_Sh,away_team_match_xG,away_team_cumulative_avg_xG,away_team_cumulative_avg_GF,away_team_cumulative_avg_GA,away_team_cumulative_avg_shots,away_team_avg_xG_last_3,away_team_avg_GF_last_3,away_team_avg_GA_last_3,away_team_avg_Sh_last_3
0,0,13/09/2025,Arsenal,0,1,11,0.5,1.500000,2.000000,0.333333,...,11,0.7,1.133333,1.333333,1.666667,10.333333,1.133333,1.333333,1.666667,10.333333
1,7,13/09/2025,Crystal Palace,3,0,5,2.6,1.433333,1.333333,0.333333,...,11,1.5,1.000000,1.666667,1.000000,10.000000,1.000000,1.666667,1.000000,10.000000
2,8,13/09/2025,Everton,3,2,10,1.9,1.433333,1.666667,1.000000,...,13,1.1,0.833333,0.000000,1.333333,11.000000,0.833333,0.000000,1.333333,11.000000
3,9,13/09/2025,Fulham,0,2,11,0.9,1.100000,0.666667,1.333333,...,10,0.7,0.966667,0.333333,1.666667,11.000000,0.966667,0.333333,1.666667,11.000000
4,17,13/09/2025,Newcastle Utd,0,0,8,0.5,0.966667,0.666667,1.000000,...,12,1.1,0.733333,0.666667,2.666667,9.000000,0.733333,0.666667,2.666667,9.000000
5,2,13/09/2025,Bournemouth,1,0,20,1.5,1.500000,1.333333,1.333333,...,11,2.2,2.033333,1.000000,1.333333,10.666667,2.033333,1.000000,1.333333,10.666667
6,24,13/09/2025,West Ham,3,0,11,2.4,1.233333,1.333333,2.666667,...,5,0.2,1.200000,1.666667,0.333333,11.000000,1.200000,1.666667,0.333333,11.000000
7,3,13/09/2025,Brentford,1,2,6,1.2,1.266667,1.000000,1.666667,...,12,2.1,2.133333,2.333333,0.333333,14.333333,2.133333,2.333333,0.333333,14.333333
8,5,14/09/2025,Burnley,2,3,6,1.2,1.033333,1.333333,2.000000,...,9,0.5,1.133333,2.666667,1.333333,11.000000,1.133333,2.666667,1.333333,11.000000
9,15,14/09/2025,Manchester City,1,2,12,1.8,1.933333,1.666667,1.333333,...,25,3.6,2.233333,1.333333,1.333333,18.666667,2.233333,1.333333,1.333333,18.666667


In [ ]:
df_gwfixfeatures = df_gwfixallmerged.drop(columns=['Date','Home Team','Away Team'])
df_gwfixfeatures.dropna(inplace=True)
df_gwfixfeatures

,Home Team encoded,home_team_match_GF,home_team_match_GA,home_team_match_Sh,home_team_match_xG,home_team_cumulative_avg_xG,home_team_cumulative_avg_GF,home_team_cumulative_avg_GA,home_team_cumulative_avg_shots,home_team_avg_xG_last_3,...,away_team_match_Sh,away_team_match_xG,away_team_cumulative_avg_xG,away_team_cumulative_avg_GF,away_team_cumulative_avg_GA,away_team_cumulative_avg_shots,away_team_avg_xG_last_3,away_team_avg_GF_last_3,away_team_avg_GA_last_3,away_team_avg_Sh_last_3
0,0,0,1,11,0.5,1.500000,2.000000,0.333333,12.333333,1.500000,...,11,0.7,1.133333,1.333333,1.666667,10.333333,1.133333,1.333333,1.666667,10.333333
1,7,3,0,5,2.6,1.433333,1.333333,0.333333,8.000000,1.433333,...,11,1.5,1.000000,1.666667,1.000000,10.000000,1.000000,1.666667,1.000000,10.000000
2,8,3,2,10,1.9,1.433333,1.666667,1.000000,9.333333,1.433333,...,13,1.1,0.833333,0.000000,1.333333,11.000000,0.833333,0.000000,1.333333,11.000000
3,9,0,2,11,0.9,1.100000,0.666667,1.333333,10.333333,1.100000,...,10,0.7,0.966667,0.333333,1.666667,11.000000,0.966667,0.333333,1.666667,11.000000
4,17,0,0,8,0.5,0.966667,0.666667,1.000000,11.333333,0.966667,...,12,1.1,0.733333,0.666667,2.666667,9.000000,0.733333,0.666667,2.666667,9.000000
5,2,1,0,20,1.5,1.500000,1.333333,1.333333,14.666667,1.500000,...,11,2.2,2.033333,1.000000,1.333333,10.666667,2.033333,1.000000,1.333333,10.666667
6,24,3,0,11,2.4,1.233333,1.333333,2.666667,11.666667,1.233333,...,5,0.2,1.200000,1.666667,0.333333,11.000000,1.200000,1.666667,0.333333,11.000000
7,3,1,2,6,1.2,1.266667,1.000000,1.666667,8.000000,1.266667,...,12,2.1,2.133333,2.333333,0.333333,14.333333,2.133333,2.333333,0.333333,14.333333
8,5,2,3,6,1.2,1.033333,1.333333,2.000000,9.000000,1.033333,...,9,0.5,1.133333,2.666667,1.333333,11.000000,1.133333,2.666667,1.333333,11.000000
9,15,1,2,12,1.8,1.933333,1.666667,1.333333,12.333333,1.933333,...,25,3.6,2.233333,1.333333,1.333333,18.666667,2.233333,1.333333,1.333333,18.666667


Encode the Home Team and Away Team, should be in the same way as in the training model

In [ ]:
#real_features = [[18,1.29,1.084,1.47,12.1,0.63333,0.33333,1.6667,8.3339,9,1.25,1.27,1.973,11.9,1.3,1.333,2.6667,9]]
#real_predictions = loaded_model.predict(real_features)

In [ ]:
realfeatures = np.array(df_gwfixfeatures)
real_predictions = loaded_model.predict(realfeatures)

In [ ]:
real_predictions

array([2, 2, 2, 2, 2, 2, 0, 0, 0, 0], dtype=int32)

In [ ]:
df_resultenc = pd.read_csv('{}/Encode tables/resultencoding.csv'.format(root))
df_resultenc

,Result,Result encoded
0,Away Win,0
1,Draw,1
2,Home Win,2


In [ ]:
df_realpred = pd.DataFrame({"Predicted Result": real_predictions})
df_realpred

,Predicted Result
0,2
1,2
2,2
3,2
4,2
5,2
6,0
7,0
8,0
9,0


In [ ]:
df_realpredenc = pd.merge(df_resultenc,df_realpred, left_on=['Result encoded'],right_on=['Predicted Result'], how='right')
df_realpredenc = df_realpredenc.drop(columns=['Result encoded','Predicted Result'])
df_realpredenc.rename(columns = {'Result':'Predicted Result'},inplace= True)
df_realpredenc

,Predicted Result
0,Home Win
1,Home Win
2,Home Win
3,Home Win
4,Home Win
5,Home Win
6,Away Win
7,Away Win
8,Away Win
9,Away Win


In [ ]:
df_gwpredictions =pd.concat([df_gwfix,df_realpredenc], axis =1)
df_gwpredictions

,Date,Home Team,Away Team,Predicted Result
0,13/09/2025,Arsenal,Nott'ham Forest,Home Win
1,13/09/2025,Crystal Palace,Sunderland,Home Win
2,13/09/2025,Everton,Aston Villa,Home Win
3,13/09/2025,Fulham,Leeds United,Home Win
4,13/09/2025,Newcastle Utd,Wolves,Home Win
5,13/09/2025,Bournemouth,Brighton,Home Win
6,13/09/2025,West Ham,Tottenham,Away Win
7,13/09/2025,Brentford,Chelsea,Away Win
8,14/09/2025,Burnley,Liverpool,Away Win
9,14/09/2025,Manchester City,Manchester Utd,Away Win


In [ ]:
df_gwpredictions.to_csv('{}/Stored predictions/Predictions 2025 09 12.csv'.format(root), encoding='utf-8', index=False)

In [ ]:
df_gwpredictions.to_csv('{}/Stored prediction latest/Latest prediction.csv'.format(root), encoding='utf-8', index=False)

Do not run program further until all gameweek results have been revealed

In [ ]:
df_gwpredictions = pd.read_csv('{}/Stored predictions/Predictions 2025 04 18.csv'.format(root))

In [ ]:
df_gwresultinit = pd.read_csv('{}/2025 26 Results/Gameweek 2 results.csv'.format(root))
df_gwresultinit = df_gwresultinit.drop(columns=['Wk', 'Day','Time', 'xG', 'xG.1', 'Attendance','Venue','Referee','Match Report','Notes'])
print(df_gwresultinit.dtypes)
df_gwresultinit

Date     object
Home     object
Score    object
Away     object
dtype: object


,Date,Home,Score,Away
0,22/08/2025,West Ham,1–5,Chelsea
1,23/08/2025,Manchester City,0–2,Tottenham
2,23/08/2025,Bournemouth,1–0,Wolves
3,23/08/2025,Brentford,1–0,Aston Villa
4,23/08/2025,Burnley,2–0,Sunderland
5,23/08/2025,Arsenal,5–0,Leeds United
6,24/08/2025,Crystal Palace,1–1,Nott'ham Forest
7,24/08/2025,Everton,2–0,Brighton
8,24/08/2025,Fulham,1–1,Manchester Utd
9,25/08/2025,Newcastle Utd,2–3,Liverpool


In [ ]:
df_gwresultinit[['homeGF','awayGF']] = df_gwresultinit['Score'].str.split('–',expand=True)
df_gwresultinit['Actual Result'] = df_gwresultinit.apply(result2, axis=1)
df_gwresult = df_gwresultinit.drop(columns=['Score','homeGF','awayGF'])
df_gwresult.rename(columns={'Home':'Home Team','Away':'Away Team'}, inplace=True)
df_gwresult

,Date,Home Team,Away Team,Actual Result
0,22/08/2025,West Ham,Chelsea,Away Win
1,23/08/2025,Manchester City,Tottenham,Away Win
2,23/08/2025,Bournemouth,Wolves,Home Win
3,23/08/2025,Brentford,Aston Villa,Home Win
4,23/08/2025,Burnley,Sunderland,Home Win
5,23/08/2025,Arsenal,Leeds United,Home Win
6,24/08/2025,Crystal Palace,Nott'ham Forest,Draw
7,24/08/2025,Everton,Brighton,Home Win
8,24/08/2025,Fulham,Manchester Utd,Draw
9,25/08/2025,Newcastle Utd,Liverpool,Away Win


In [ ]:
df_predactualresult = pd.merge(df_gwpredictions,df_gwresult, on=['Date','Home Team','Away Team'])
df_predactualresult

,Date,Home Team,Away Team,Predicted Result,Actual Result
0,22/08/2025,West Ham,Chelsea,Away Win,Away Win
1,23/08/2025,Manchester City,Tottenham,Draw,Away Win
2,23/08/2025,Bournemouth,Wolves,Home Win,Home Win
3,23/08/2025,Brentford,Aston Villa,Away Win,Home Win
4,23/08/2025,Burnley,Sunderland,Away Win,Home Win
5,23/08/2025,Arsenal,Leeds United,Away Win,Home Win
6,24/08/2025,Crystal Palace,Nott'ham Forest,Away Win,Draw
7,24/08/2025,Everton,Brighton,Home Win,Home Win
8,24/08/2025,Fulham,Manchester Utd,Away Win,Draw
9,25/08/2025,Newcastle Utd,Liverpool,Away Win,Away Win


In [ ]:
df_predactualresult.to_csv('{}/Stored predictions_results/Predictions results 2025 08 22.csv'.format(root), encoding='utf-8', index=False)

In [ ]:
#df_predactualresult = pd.read_csv('{}/Stored predictions_results/Predictions results 2024 08 16.csv'.format(root))

In [ ]:
df_predactual_master_init = pd.read_csv('{}/Stored predictions_results_master/Predictions results_master.csv'.format(root))

In [ ]:
df_predact_master = pd.concat([df_predactual_master_init, df_predactualresult], axis=0, ignore_index=True)
df_predact_master

#df_allteam = pd.concat(team_temp, axis=0, ignore_index=True)
#df_allteam

,Date,Home Team,Away Team,Predicted Result,Actual Result
0,11/08/2023,Burnley,Manchester City,Away Win,Away Win
1,12/08/2023,Arsenal,Nott'ham Forest,Home Win,Home Win
2,12/08/2023,Everton,Fulham,Home Win,Away Win
3,12/08/2023,Sheffield Utd,Crystal Palace,Away Win,Away Win
4,12/08/2023,Brighton,Luton Town,Home Win,Home Win
...,...,...,...,...,...
764,23/08/2025,Arsenal,Leeds United,Away Win,Home Win
765,24/08/2025,Crystal Palace,Nott'ham Forest,Away Win,Draw
766,24/08/2025,Everton,Brighton,Home Win,Home Win
767,24/08/2025,Fulham,Manchester Utd,Away Win,Draw


In [ ]:
df_predact_master.to_csv('{}/Stored predictions_results_master/Predictions results_master.csv'.format(root), encoding='utf-8', index=False)